    lstm的python实现

    前面生成数据的部分和rnn数据处理部分相同
    样本数据来自于reddit评论，使用nltk处理后作为训练数据

In [4]:
import csv
import nltk
import itertools
import numpy as np
import datetime

#定义一些基本的常量
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
start_token = "SENTENCE_START"
end_token = "SENTENCE_END"

#处理数据，这里的数据是从reddit上面下载的英文的评论，预先使用nltk工具处理后，
#得到原始的训练数据集
file_path = "reddit-comments-2015-08.csv"
tokenized = []
original = []
with open(file_path, 'r') as f:
    #读取内容
    reader = csv.reader(f, skipinitialspace=True)
    for x in reader:
        #每个句子前后都加上开始和结束标记，然后再使用nltk分词
        sentences = nltk.sent_tokenize(x[0].lower())
        for sentence in sentences:
            original.append(sentence)
            sentence = start_token + " " + sentence + " " + end_token
            tokenized.append(nltk.word_tokenize(sentence))
    f.close()

#统计词频
word_freq = nltk.FreqDist(itertools.chain(*tokenized))

#获取常用词，然后构建词频向量
vocab = word_freq.most_common(vocabulary_size - 1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([w, i] for i, w in enumerate(index_to_word))

print("vacabulary size {}".format(vocabulary_size))
print("The least frequent word in our vocabulary is '{}' and appeared times {}".format(vocab[-1][0], vocab[-1][1]))

#将所有不在词汇表中的词替换为unknown token
for i, sent in enumerate(tokenized):
    tokenized[i] = [w if w in word_to_index else unknown_token for w in sent]
    
#创建训练数据
#训练的主要目的是预测下一个词，所以x的数据是对应y数据的前一个词
x_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized])

print(x_train[1])
print(y_train[1])

vacabulary size 8000
The least frequent word in our vocabulary is 'documentary' and appeared times 10
[0, 6, 3494, 7, 155, 795, 25, 222, 8, 32, 20, 202, 4954, 350, 91, 6, 66, 207, 5, 2]
[6, 3494, 7, 155, 795, 25, 222, 8, 32, 20, 202, 4954, 350, 91, 6, 66, 207, 5, 2, 1]


In [12]:
#先定义一下常用的激活函数
def softmax(x):
    x = np.array(x)
    max_x = np.max(x)
    return np.exp(x - max_x) / np.sum(np.exp(x - max_x))
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

In [13]:
#初始化一些参数
data_dim = vocabulary_size
hidden_dim = 100
learning_rate = 0.005
def init_wb():
    wh = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim), (hidden_dim, hidden_dim))
    wx = np.random.uniform(-np.sqrt(1.0 / data_dim), np.sqrt(1.0 / data_dim), (hidden_dim, data_dim))
    b = np.random.uniform(-np.sqrt(1.0 / data_dim), np.sqrt(1.0 / data_dim), (hidden_dim, 1))
    return wh, wx, b

#初始化权重向量，根据公式有
#1.遗忘门参数,whf,wxf,bf
#2.输入门参数,whi,whi,bi,和新信息参数wha,wxa,ba
#3.输出门参数,who,wxo,bo
whf, wxf, bf = init_wb()
whi, wxi, bi = init_wb()
wha, wxa, ba = init_wb()
who, wxo, bo = init_wb()

#输出权重和偏置，y^ = wy*o+wb
wy = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim), (data_dim, hidden_dim))
by = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim), (data_dim, 1))

#将这些参数封装在对象中
variables = {
    "whf" : whf, "wxf" : wxf, "bf" : bf, "whi" : whi, "wxi" : wxi, "bi" : bi,
    "wha" : wha, "wxa" : wxa, "ba" : ba,
    "who" : who, "wxo" : wxo, "bo" : bo,
    "wy" : wy, "by" : by
}

In [27]:
#初始化一些状态向量
def init_s(T):
    #输入门it,新信息ast，遗忘门ft，输出门ot,ht,输出ys,新细胞状态ct
    ist = np.array([np.zeros((hidden_dim, 1))] * (T + 1))
    cst = np.array([np.zeros((hidden_dim, 1))] * (T + 1))
    fst = np.array([np.zeros((hidden_dim, 1))] * (T + 1))
    ost = np.array([np.zeros((hidden_dim, 1))] * (T + 1))                                   
    hst = np.array([np.zeros((hidden_dim, 1))] * (T + 1))
    yst = np.array([np.zeros((data_dim, 1))] * T)
    ast = np.array([np.zeros((hidden_dim, 1))] * (T + 1)) 
    return {"ist" : ist, "cst" : cst, "fst":fst, "ost":ost, "hst":hst, "yst":yst, "ast":ast}

#前向传播根据x，计算各种状态向量的值
def forward(x):
    T = len(x)
    status = init_s(T)
#     print("data_dim:{},hidden_dim:{}".format(data_dim, hidden_dim))
    for t in range(T):
        hst_pre = np.array(status["hst"][t - 1]).reshape(-1, 1)
        
        #ist
        status["ist"][t] = cal_gate(variables['whi'], variables['wxi'], variables['bi'], x[t], hst_pre, sigmoid)
        #fst
        status["fst"][t] = cal_gate(variables['whf'], variables['wxf'], variables['bf'], x[t], hst_pre, sigmoid)
        #ast
        status["ast"][t] = cal_gate(variables['wha'], variables['wxa'], variables['ba'], x[t], hst_pre, tanh)
        #ost
        status["ost"][t] = cal_gate(variables['who'], variables['wxo'], variables['bo'], x[t], hst_pre, sigmoid)
        #cst
        status["cst"][t] = status["fst"][t] * status["cst"][t-1] +  status["ist"][t] * status["ast"][t]
        #hst
        status["hst"][t] = status["ost"][t] * tanh(status["cst"][t])
        #yst
        status["yst"][t] = softmax(variables['wy'].dot(status["hst"][t]) + variables['by'])
        
        
    return status
        
def cal_gate(wh, wx, b, x, hst_pre, activation):
    return activation(wh.dot(hst_pre) + wx[:, x].reshape(-1, 1) + b)

In [28]:
#预测函数值
def predict(x):
    status = forward(x)
    pre_y = np.argmax(status["yst"].reshape(len(x), -1), axis=1)
    return pre_y

In [29]:
#测试预测函数
predict(x_train[0])

array([2456, 2456])

In [30]:
#计算损失,定义为softmax输出函数,损失为交叉熵损失
# 计算损失， softmax交叉熵损失函数， (x,y)为多个样本
def loss(x, y):
    cost = 0        
    for i in range(len(y)):
        status = forward(x[i])
        # 取出 y[i] 中每一时刻对应的预测值
        pre_yi = status['yst'][range(len(y[i])), y[i]]
        cost -= np.sum(np.log(pre_yi))

    # 统计所有y中词的个数, 计算平均损失
    N = np.sum([len(yi) for yi in y])
    ave_loss = cost / N

    return ave_loss

In [31]:
print("random loss:{}".format(loss(x_train[:10], y_train[:10])))

random loss:8.996643532327656


    反向传播计算梯度

In [32]:
# 初始化偏导数 dwh, dwx, db
def init_wb_grad():
    dwh = np.zeros(variables['whi'].shape)
    dwx = np.zeros(variables['wxi'].shape)
    db = np.zeros(variables['bi'].shape)

    return dwh, dwx, db

In [35]:
#反向传播计算梯度
def bptt(x, y):
    dwhi, dwxi, dbi = init_wb_grad()
    dwhf, dwxf, dbf = init_wb_grad()                           
    dwho, dwxo, dbo = init_wb_grad()
    dwha, dwxa, dba = init_wb_grad()
    dwy, dby = np.zeros(variables['wy'].shape), np.zeros(variables['by'].shape)

    # 初始化 delta_ct，因为后向传播过程中，此值需要累加
    delta_ct = np.zeros((hidden_dim, 1))

    # 前向计算
    stats = forward(x)
    # 目标函数对输出 y 的偏导数
    delta_o = stats['yst']
    delta_o[np.arange(len(y)), y] -= 1

    for t in np.arange(len(y))[::-1]:
        # 输出层wy, by的偏导数，由于所有时刻的输出共享输出权值矩阵，故所有时刻累加
        dwy += delta_o[t].dot(stats['hst'][t].reshape(1, -1))  
        dby += delta_o[t]

        # 目标函数对隐藏状态的偏导数
        delta_ht = variables['wy'].T.dot(delta_o[t])

        # 各个门及状态单元的偏导数
        delta_ot = delta_ht * tanh(stats['cst'][t])
        delta_ct += delta_ht * stats['ost'][t] * (1-tanh(stats['cst'][t])**2)
        delta_it = delta_ct * stats['ast'][t]
        delta_ft = delta_ct * stats['cst'][t-1]
        delta_at = delta_ct * stats['ist'][t]

        delta_at_net = delta_at * (1-stats['ast'][t]**2)
        delta_it_net = delta_it * stats['ist'][t] * (1-stats['ist'][t])
        delta_ft_net = delta_ft * stats['fst'][t] * (1-stats['fst'][t])
        delta_ot_net = delta_ot * stats['ost'][t] * (1-stats['ost'][t])

        # 更新各权重矩阵的偏导数，由于所有时刻共享权值，故所有时刻累加
        dwhf, dwxf, dbf = cal_grad_delta(dwhf, dwxf, dbf, delta_ft_net, stats['hst'][t-1], x[t])                              
        dwhi, dwxi, dbi = cal_grad_delta(dwhi, dwxi, dbi, delta_it_net, stats['hst'][t-1], x[t])                              
        dwha, dwxa, dba = cal_grad_delta(dwha, dwxa, dba, delta_at_net, stats['hst'][t-1], x[t])            
        dwho, dwxo, dbo = cal_grad_delta(dwho, dwxo, dbo, delta_ot_net, stats['hst'][t-1], x[t])

    return [dwhf, dwxf, dbf, 
            dwhi, dwxi, dbi, 
            dwha, dwxa, dba, 
            dwho, dwxo, dbo, 
            dwy, dby]
    pass

#计算权重偏导数的公共方法
def cal_grad_delta(dwh, dwx, db, delta_net, ht_pre, x):
    dwh += delta_net * ht_pre
    dwx += delta_net * x
    db += delta_net

    return dwh, dwx, db

In [36]:
#更新梯度,
def sgd_step(x, y, learning_rate):
    dwhf, dwxf, dbf, dwhi, dwxi, dbi, dwha, dwxa, dba, dwho, dwxo, dbo, dwy, dby = bptt(x, y)
#     print(dwhf[0, 0])
    
    #更新权重矩阵
    variables['whf'], variables['wxf'], variables['bf'] = update_wb(learning_rate, variables['whf'], variables['wxf'], variables['bf'], dwhf, dwxf, dbf)
    variables['whi'], variables['wxi'], variables['bi'] = update_wb(learning_rate, variables['whi'], variables['wxi'], variables['bi'], dwhi, dwxi, dbi) 
    variables['wha'], variables['wxa'], variables['ba'] = update_wb(learning_rate, variables['wha'], variables['wxa'], variables['ba'], dwha, dwxa, dba) 
    variables['who'], variables['wxo'], variables['bo'] = update_wb(learning_rate, variables['who'], variables['wxo'], variables['bo'], dwho, dwxo, dbo) 
    variables['wy'] = variables['wy'] - learning_rate * dwy
    variables['by'] = variables['by'] - learning_rate * dby
    
#更新权重矩阵的公共方法
def update_wb(learning_rate, wh, wx, b, dwh, dwx, db):
    wh -= learning_rate * dwh
    wx -= learning_rate * dwx
    b -= learning_rate * db
    return wh, wx, b

In [37]:
#训练lstm
def train(x_train, y_train, learning_rate=0.005, n_epoch=5):
    losses = []
    num_examples = 0.05
    
    for epoch in range(n_epoch):
        for i in range(len(x_train)):
            sgd_step(x_train[i], y_train[i], learning_rate)
            num_examples += 1
        
        cost = loss(x_train, y_train)
        losses.append(cost)
        print("epoch:{},loss:{}".format(epoch + 1, cost))
        if len(losses) > 1 and losses[-1] > losses[-2]:
            learning_rate *= 0.5
            print("decrease learning_rate to {}".format(learning_rate))

In [ ]:
train(x_train[:200], y_train[:200], n_epoch=3)

epoch:1,loss:6.327918914206463
epoch:2,loss:6.075115714313507
